In [50]:
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from numpy import linspace, dot
import scipy.signal as signal
from numpy.linalg import svd
from collections import defaultdict
from matplotlib.pyplot import plot
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
NEGATIVE_LABEL = "control"
POSITIVE_LABEL = 'injured'

def get_filepaths_and_meta(csv_folder):
    ret = [extract_meta(f) for f in os.listdir(csv_folder) if f.endswith('.csv')]
    for x in ret:
        x['filepath'] = os.path.join(csv_folder, x['filepath'])
    return ret

def read_and_process_csv(csv_fpath):
    df = pd.read_csv(csv_fpath)
    del df[df.columns[-1]]
    del df['Unnamed: 0']
    return df

def pca(df):
    num_pixels = len(df)
    df_standardized = (df - df.mean()) / df.std()
    df_standardized = df_standardized.fillna(0)
    covariance_mat = np.cov(df_standardized.values)
    # img = (covariance_mat-covariance_mat.min()) *255 / (covariance_mat.max()-covariance_mat.min())
    # plt.imshow(img)
    # plt.show()
    return svd(covariance_mat, full_matrices=False)

def dim_reduction(df):
    U, S, Vh = pca(df)
    total_s = sum(S)
    cumsum = 0
    for i in range(len(S)):
        cumsum += S[i]/total_s
        if cumsum > .99:
            break;
    return dot(dot(U,S),Vh)

def select_tissue_region(df_feature_set):
    [U, S, V] = pca(df_feature_set)
    S_DIAG = np.diag(S)
    dim_red_data = dot(U[:, :2], S_DIAG[:2, :2])
    kmeans = KMeans(n_clusters=3, n_init=15)
    kmeans.fit(df_feature_set)
    y_kmeans = kmeans.predict(df_feature_set)
    df_clusters = df_feature_set
    df_clusters['clusters'] = y_kmeans
    unique, counts = np.unique(y_kmeans, return_counts=True)
    cluster_counts = dict(zip(unique, counts))
    white_space_key = max(cluster_counts, key=cluster_counts.get)
    df_remove_white_space = df_clusters[df_clusters['clusters'] != white_space_key]

    del df_remove_white_space['clusters']
    return df_remove_white_space


def kmeans(df, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(df.values)
    return kmeans

def graph_ndim_kmeans(df, n_clusters):
    U, S, Vh = pca(df)
    xs = dot(S[0],Vh[0,:])
    ys = dot(S[1],Vh[1,:])
    labels, centers = kmeans(df, n_clusters)
    plt.scatter(xs, ys, c=labels, s=20, cmap='viridis')
    # plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)
    plt.title('K-means of Pixel MS data')
    plt.xlabel('Singular Vector 1')
    plt.ylabel('Singular Vector 2')
    plt.show()

def day_groups(files_and_meta):
    ret = defaultdict(lambda: [])
    for fam in files_and_meta:
        ret[(fam['label'], fam['day'])].append((fam, read_and_process_csv(fam['filepath'])))
    return ret

def extract_meta(fpath):
    regex_search = re.search(r'(.*)_Day(\d\d)_(\d\d)_(\d\d)x(\d\d)_', fpath)
    label, day, experiment, size_y, size_x = regex_search.groups()
    assert label.lower() in [NEGATIVE_LABEL, POSITIVE_LABEL]
    return {
        'filepath': fpath,
        'label': 1 if label.lower() == POSITIVE_LABEL else -1,
        'day': int(day),
        'experiment': int(experiment),
        'x': int(size_x),
        'y': int(size_y)
    }

def filter_meta_day_exp(day, experiment):
    experiments = get_filepaths_and_meta(input_folder);
    ret = filter(lambda x: x['day'] == day and x['experiment'] == experiment,experiments)
    return list(ret)

def get_x_y(day, experiment):
    meta = filter_meta_day_exp(day, experiment)
    print(meta[0]['filepath'])
    print(meta[1]['filepath'])
    x0 = select_tissue_region(read_and_process_csv(meta[0]['filepath'])).values
    y0 = np.ones(len(x0)) * meta[0]['label']
    x1 = select_tissue_region(read_and_process_csv(meta[1]['filepath'])).values
    y1 = np.ones(len(x1)) * meta[1]['label']
    return np.vstack((x0,x1)), np.append(y0,y1)

In [70]:
input_folder = r"\\wfs1\users$\mccloskey\Downloads\csvData-20190806T210426Z-001\csvData\aggregates (bin=1000)"
X, y = get_x_y()
X_standardized = (X - X.mean()) / X.std()

\\wfs1\users$\mccloskey\Downloads\csvData-20190806T210426Z-001\csvData\aggregates (bin=1000)\Control_Day07_02_30x30_aggregated.csv
\\wfs1\users$\mccloskey\Downloads\csvData-20190806T210426Z-001\csvData\aggregates (bin=1000)\Injured_Day07_02_31x30_aggregated.csv


In [ ]:
def dim_reduction(df):
    U, S, Vh = pca(df)
    total_s = sum(S)
    cumsum = 0
    for i in range(len(S)):
        cumsum += S[i]/total_s
        if cumsum > .99:
            break;
    return dot(dot(U,S),Vh)

In [71]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
lasso = linear_model.LinearRegression()
lasso.fit(X_standardized,y)
score = cross_val_score(lasso, X_standardized, y, cv=20)

In [73]:
X2, y2 = get_x_y(7,1)
X2_standardized = (X2 - X2.mean()) / X2.std()
y2_pred = np.sign(lasso.predict(X2_standardized))
f1_score(y2, y2_pred)

\\wfs1\users$\mccloskey\Downloads\csvData-20190806T210426Z-001\csvData\aggregates (bin=1000)\Control_Day07_01_29x29_aggregated.csv
\\wfs1\users$\mccloskey\Downloads\csvData-20190806T210426Z-001\csvData\aggregates (bin=1000)\Injured_Day07_01_38x35_aggregated.csv


0.3415492957746479

In [74]:
X2, y2 = get_x_y(3,2)
X2_standardized = (X2 - X2.mean()) / X2.std()
X

\\wfs1\users$\mccloskey\Downloads\csvData-20190806T210426Z-001\csvData\aggregates (bin=1000)\Control_Day03_02_35x34_aggregated.csv
\\wfs1\users$\mccloskey\Downloads\csvData-20190806T210426Z-001\csvData\aggregates (bin=1000)\Injured_Day03_02_37x33_aggregated.csv


array([[502755.6875    ,      0.        ,  50773.81640625, ...,
             0.        ,      0.        ,      0.        ],
       [422178.84375   ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ],
       [565333.3046875 ,      0.        ,  53997.140625  , ...,
             0.        ,      0.        ,      0.        ],
       ...,
       [445135.703125  ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ],
       [494587.578125  ,      0.        ,  52764.4921875 , ...,
             0.        ,      0.        ,      0.        ],
       [441153.609375  ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ]])

In [26]:
X_standardized.shape

(493, 249)

In [19]:
X_standardized

array([[ 2.43915544, -0.46934391,  0.81051543, ..., -0.46934391,
        -0.46934391, -0.46934391],
       [ 2.15477345, -0.46934391,  0.63016466, ..., -0.46934391,
        -0.46934391, -0.46934391],
       [ 2.0010301 , -0.46934391,  0.57276638, ..., -0.46934391,
        -0.46934391, -0.46934391],
       ...,
       [ 1.68272576, -0.46934391,  0.56165441, ..., -0.46934391,
        -0.46934391, -0.46934391],
       [ 0.93495153, -0.46934391,  0.21005842, ..., -0.46934391,
        -0.46934391, -0.46934391],
       [ 1.63850939, -0.46934391,  0.55334769, ..., -0.46934391,
        -0.46934391, -0.46934391]])

In [27]:
y.shape

(244,)